In [1]:

import sys

sys.path.append("..")
from data.database import SessionLocal
from data.data_models import Products

from tqdm import tqdm

In [2]:

import os
import requests
from openai import OpenAI
from sqlalchemy.orm import Session
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")

client = OpenAI()
def generate_and_save_image(prompt, filename):
    # Check if the image already exists
    if not os.path.exists(filename):
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )

        image_url = response.data[0].url
        image_response = requests.get(image_url)
        
        # Ensure the directory exists
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        with open(filename, 'wb') as f:
            f.write(image_response.content)
    else:
        print(f"Image already exists: {filename}")

def process_restaurants(db: Session):
    products = db.query(Products).all()
    print("Generating restaurant images...")
    print(products)
    for product in tqdm(products):
        try:
            prompt = f"A high-quality, realistic image thumbnail of a {product.name} by {product.brand}. The product is displayed on a clean, white background with soft shadows to make it stand out. The image is crisp and clear, showcasing the product's details, texture, and branding elements. This image will be used as a thumbnail for an online store, so the product should be centered, well-lit, and visually appealing."
            filename = os.path.join('../../../frontend/src/assets/', product.image)
            generate_and_save_image(prompt, filename)
        except Exception as e:
            print(e)


# Get the database session
db = SessionLocal()

try:
    # Process restaurants and foods
    process_restaurants(db)
except Exception as e:
    print(e)

finally:
    # Close the session
    db.close()

Generating restaurant images...
[<data.data_models.Products object at 0x000001E768AB0370>, <data.data_models.Products object at 0x000001E768AB0310>, <data.data_models.Products object at 0x000001E768AB03A0>, <data.data_models.Products object at 0x000001E768AB0400>, <data.data_models.Products object at 0x000001E768AB0430>, <data.data_models.Products object at 0x000001E768AB04C0>, <data.data_models.Products object at 0x000001E768AB0550>, <data.data_models.Products object at 0x000001E768AB05E0>, <data.data_models.Products object at 0x000001E768AB0670>, <data.data_models.Products object at 0x000001E768AB0700>, <data.data_models.Products object at 0x000001E768AB0790>, <data.data_models.Products object at 0x000001E768AB0820>, <data.data_models.Products object at 0x000001E768AB08B0>, <data.data_models.Products object at 0x000001E768AB0940>, <data.data_models.Products object at 0x000001E768AB09D0>, <data.data_models.Products object at 0x000001E768AB0A60>, <data.data_models.Products object at 0x

100%|██████████| 25/25 [07:16<00:00, 17.48s/it]
